In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt

import seaborn as sns

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image

from tensorflow.keras.models import load_model

import sys
sys.path.append('../input/util.py')

In [ ]:
#prepare files and labels using get_dummies
df_train=pd.read_csv('Train_Data.csv',header=None)

df_train.columns=['FileNames','Labels']

train_df=pd.DataFrame()

for c in range(0,len(df_train)):
    train_df['FileNames']=df_train['FileNames'].apply(lambda x: (x.replace("'",'')))
    train_df['Labels']=df_train['Labels'].apply(lambda x: (x.replace("'",'')))

train_labels=train_df['Labels']
train_new_label=pd.get_dummies(train_labels)

train_new_df = pd.concat([train_df['FileNames'], train_new_label], axis=1)
train_new_df['COVID']=train_new_df['COVID'].astype(np.float32)
train_new_df['Normal']=train_new_df['Normal'].astype(np.float32)
train_new_df['Viral Pneumonia']=train_new_df['Viral Pneumonia'].astype(np.float32)
train_new_df.head()

# Oversampling of minority classes in the training set

In [ ]:
#oversampling of the traning dataset

df_train=train_new_df

#Extract only COVID data
is_COVID=df['COVID']==1
Co_df=df[is_COVID]
Co_df.head
Co_df.shape

#Extract only Pneumonia data
is_P=df['Viral Pneumonia']==1
P_df=df[is_P]
P_df.head
P_df.shape

#Extract only Normal data
is_N=df['Normal']==1
N_df=df[is_N]
N_df.head
N_df.shape

##Oversample Covid Data with 2.8x
sampled_Co_df=Co_df.sample(frac=2.8, replace=True, random_state=1)

sampled_Co_df.head()

#oversample pneumonia data with 7.5x
sampled_P_df=P_df.sample(frac=7.5, replace=True, random_state=1)

sampled_P_df.head()

#Concatenate all class togehter into one dataframe
Oversampled_train_df=pd.concat([sampled_N_df,sampled_P_df])
Oversampled_train_df=pd.concat([Oversampled_train_df,Co_df])


In [ ]:
#prepare files and labels using get_dummies
df_test=pd.read_csv('Test_Data.csv',header=None)

df_test.columns=['FileNames','Labels']

test_df=pd.DataFrame()

for c in range(0,len(df_test)):
    test_df['FileNames']=df_test['FileNames'].apply(lambda x: (x.replace("'",'')))
    test_df['Labels']=df_test['Labels'].apply(lambda x: (x.replace("'",'')))

test_labels=test_df['Labels']
test_new_label=pd.get_dummies(test_labels)

test_new_df = pd.concat([test_df['FileNames'], test_new_label], axis=1)
test_new_df['COVID']=test_new_df['COVID'].astype(np.float32)
test_new_df['Normal']=test_new_df['Normal'].astype(np.float32)
test_new_df['Viral Pneumonia']=test_new_df['Viral Pneumonia'].astype(np.float32)
test_new_df.head()

In [ ]:
#prepare files and labels using get_dummies
df_valid=pd.read_csv('Validate_Data.csv',header=None)

df_valid.columns=['FileNames','Labels']

validt_df=pd.DataFrame()

for c in range(0,len(df_valid)):
    valid_df['FileNames']=df_valid['FileNames'].apply(lambda x: (x.replace("'",'')))
    valid_df['Labels']=df_valid['Labels'].apply(lambda x: (x.replace("'",'')))

valid_labels=valid_df['Labels']
valid_new_label=pd.get_dummies(valid_labels)

valid_new_df = pd.concat([valid_df['FileNames'], valid_new_label], axis=1)
valid_new_df['COVID']=valid_new_df['COVID'].astype(np.float32)
valid_new_df['Normal']=valid_new_df['Normal'].astype(np.float32)
valid_new_df['Viral Pneumonia']=valid_new_df['Viral Pneumonia'].astype(np.float32)
valid_new_df.head()

In [ ]:
def get_train_generator(df, image_dir, x_col, y_cols, shuffle=True, batch_size=8, seed=1, target_w = 320, target_h = 320):
  
    print("getting train generator...") 
    # normalize images
    image_generator = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization= True)
    
    # flow from directory with specified batch size
    # and target image size
    generator = image_generator.flow_from_dataframe(
            dataframe=df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            target_size=(target_w,target_h))
    
    return generator

In [ ]:
def get_test_and_valid_generator(valid_df, test_df, train_df, image_dir, x_col, y_cols, sample_size=100, batch_size=8, seed=1, target_w = 320, target_h = 320):

    print("getting train and valid generators...")
    # get generator to sample dataset
    raw_train_generator = ImageDataGenerator().flow_from_dataframe(
        dataframe=train_df, 
        directory=IMAGE_DIR, 
        x_col=x_col, 
        y_col=labels, 
        class_mode="raw", 
        batch_size=sample_size, 
        shuffle=True, 
        target_size=(target_w, target_h))
    
    # get data sample
    batch = raw_train_generator.next()
    data_sample = batch[0]

    # use sample to fit mean and std for test set generator
    image_generator = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization= True)
    
    # fit generator to sample from training data
    image_generator.fit(data_sample)

    # get test generator
    valid_generator = image_generator.flow_from_dataframe(
            dataframe=valid_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            seed=seed,
            target_size=(target_w,target_h))

    test_generator = image_generator.flow_from_dataframe(
            dataframe=test_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            seed=seed,
            target_size=(target_w,target_h))
    return valid_generator, test_generator

In [ ]:
#use oversampled training set as the training data here
labels=['COVID','Normal','Viral Pneumonia']

IMAGE_DIR = "../input/segmented_Xrays/"
train_generator = get_train_generator(Oversampled_train_df, IMAGE_DIR, "FileNames", labels)
valid_generator, test_generator= get_test_and_valid_generator(valid_new_df, test_new_df, Oversampled_train_df, IMAGE_DIR,  "FileNames", labels)

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model  =EfficientNetB7(weights='imagenet',include_top=False)

#base_model = DenseNet121(weights='../input/library/densenet.hdf5', include_top=False)

x = base_model.output

# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)

# and a logistic layer
predictions = Dense(len(labels), activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
#train the net
history = model.fit(train_generator, 
                    batch_size=10,
                    validation_data=valid_generator,
                    steps_per_epoch=100, 
                    validation_steps=25, 
                    epochs = 100)

plt.plot(history.history['loss'])
plt.ylabel("loss")
plt.xlabel("epoch")
plt.title("Training Loss Curve")
plt.show()

plt.plot(history.history['accuracy'])
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.title("Training Accuracy Curve")
plt.show()

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Trained_EfficientNetB7_Oversampling.h5")
print("Saved model to disk")

In [ ]:
#Prediction of new data in test data
predicted_vals = model.predict(test_generator, steps = len(test_generator))

In [ ]:
#this code was derived from https://www.coursera.org/learn/ai-for-medical-diagnosis
import random

import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from sklearn.metrics import roc_auc_score, roc_curve
from tensorflow.compat.v1.logging import INFO, set_verbosity
import tensorflow as tf #I added
random.seed(a=None, version=2)

set_verbosity(INFO)



def get_mean_std_per_batch(image_path, df, H=320, W=320):
    sample_data = []
    for idx, img in enumerate(df.sample(100)["FileNames"].values):
        # path = image_dir + img
        sample_data.append(
            np.array(image.load_img(image_path, target_size=(H, W))))

    mean = np.mean(sample_data[0])
    std = np.std(sample_data[0])
    return mean, std


def load_image(img, image_dir, df, preprocess=True, H=320, W=320):
    """Load and preprocess image."""
    img_path = image_dir + img
    mean, std = get_mean_std_per_batch(img_path, df, H=H, W=W)
    x = image.load_img(img_path, target_size=(H, W))
    if preprocess:
        x -= mean
        x /= std
        x = np.expand_dims(x, axis=0)
    return x

def grad_cam(input_model, image, cls, layer_name, H=320, W=320):
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.output[0, cls]
    conv_output = input_model.get_layer(layer_name).output

    grads = K.gradients(y_c, conv_output)[0]

    gradient_function = K.function([input_model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # Process CAM
    cam = cv2.resize(cam, (W, H), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

def compute_gradcam(model, img, image_dir, df, labels, selected_labels,
                    layer_name='bn'):
    preprocessed_input = load_image(img, image_dir, df)
    predictions = model.predict(preprocessed_input)

    print("Loading original image")
    plt.figure(figsize=(15, 10))
    plt.subplot(151)
    plt.title("Original")
    plt.axis('off')
    plt.imshow(load_image(img, image_dir, df, preprocess=False), cmap='gray')

    j = 1
    for i in range(len(labels)):
        if labels[i] in selected_labels:
            print(f"Generating gradcam for class {labels[i]}")
            gradcam = grad_cam(model, preprocessed_input, i, layer_name)
            plt.subplot(151 + j)
            plt.title(f"{labels[i]}: p={predictions[0][i]:.3f}")
            plt.axis('off')
            plt.imshow(load_image(img, image_dir, df, preprocess=False),
                       cmap='gray')
            plt.imshow(gradcam, cmap='jet', alpha=min(0.5, predictions[0][i]))
            j += 1
def get_roc_curve(labels, predicted_vals, generator):
    auc_roc_vals = []
    for i in range(len(labels)):
        try:
            gt = generator.labels[:, i]
            pred = predicted_vals[:, i]
            auc_roc = roc_auc_score(gt, pred)
            auc_roc_vals.append(auc_roc)
            fpr_rf, tpr_rf, _ = roc_curve(gt, pred)
            plt.figure(1, figsize=(10, 10))
            plt.plot([0, 1], [0, 1], 'k--')
            plt.plot(fpr_rf, tpr_rf,
                     label=labels[i] + " (" + str(round(auc_roc, 3)) + ")")
            plt.xlabel('False positive rate')
            plt.ylabel('True positive rate')
            plt.title('ROC curve')
            plt.legend(loc='best')
        except:
            print(
                f"Error in generating ROC curve for {labels[i]}. "
                f"Dataset lacks enough examples."
            )
    plt.show()
    return auc_roc_vals

In [ ]:
#plot ROC curve
labels=['COVID','Normal','Viral Pneumonia']
auc_rocs = get_roc_curve(labels, predicted_vals, test_generator)

In [ ]:
valid_results = model.predict(valid_generator, steps = len(valid_generator))
test_results = model.predict(test_generator, steps = len(test_generator))


# the labels for prediction values in our dataset
pred_labels = [l + "_pred" for l in labels]

print(valid_results.shape)

results_valid=pd.DataFrame()
results_valid[labels]=valid_new_df[labels]


for i,x in enumerate(pred_labels):
    results_valid[x]=valid_results[:,i]

results_test=pd.DataFrame()
results_test[labels]=test_new_df[labels]


for i,x in enumerate(pred_labels):
    results_test[x]=test_results[:,i]
print(results_test.head()) 

In [ ]:
y = test_df[labels].values
pred = test_result[pred_labels].values

plt.xticks(rotation=90)
plt.bar(x = labels, height= y.sum(axis=0));

In [ ]:
#function for evaluation metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import (
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)


def get_true_pos(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == True) & (y == 1))


def get_true_neg(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == False) & (y == 0))


def get_false_neg(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == False) & (y == 1))


def get_false_pos(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == True) & (y == 0))


def get_performance_metrics(y, pred, class_labels, tp=get_true_pos,
                            tn=get_true_neg, fp=get_false_pos,
                            fn=get_false_neg,
                            acc=None, prevalence=None, spec=None,
                            sens=None, ppv=None, npv=None, auc=None, f1=None,
                            thresholds=[]):
    if len(thresholds) != len(class_labels):
        thresholds = [.5] * len(class_labels)

    columns = ["", "TP", "TN", "FP", "FN", "Accuracy", "Prevalence",
               "Sensitivity",
               "Specificity", "PPV", "NPV", "AUC", "F1", "Threshold"]
    df = pd.DataFrame(columns=columns)
    for i in range(len(class_labels)):
        df.loc[i] = [""] + [0] * (len(columns) - 1)
        df.loc[i][0] = class_labels[i]
        df.loc[i][1] = round(tp(y[:, i], pred[:, i]),
                             3) if tp != None else "Not Defined"
        df.loc[i][2] = round(tn(y[:, i], pred[:, i]),
                             3) if tn != None else "Not Defined"
        df.loc[i][3] = round(fp(y[:, i], pred[:, i]),
                             3) if fp != None else "Not Defined"
        df.loc[i][4] = round(fn(y[:, i], pred[:, i]),
                             3) if fn != None else "Not Defined"
        df.loc[i][5] = round(acc(y[:, i], pred[:, i], thresholds[i]),
                             3) if acc != None else "Not Defined"
        df.loc[i][6] = round(prevalence(y[:, i]),
                             3) if prevalence != None else "Not Defined"
        df.loc[i][7] = round(sens(y[:, i], pred[:, i], thresholds[i]),
                             3) if sens != None else "Not Defined"
        df.loc[i][8] = round(spec(y[:, i], pred[:, i], thresholds[i]),
                             3) if spec != None else "Not Defined"
        df.loc[i][9] = round(ppv(y[:, i], pred[:, i], thresholds[i]),
                             3) if ppv != None else "Not Defined"
        df.loc[i][10] = round(npv(y[:, i], pred[:, i], thresholds[i]),
                              3) if npv != None else "Not Defined"
        df.loc[i][11] = round(auc(y[:, i], pred[:, i]),
                              3) if auc != None else "Not Defined"
        df.loc[i][12] = round(f1(y[:, i], pred[:, i] > thresholds[i]),
                              3) if f1 != None else "Not Defined"
        df.loc[i][13] = round(thresholds[i], 3)

    df = df.set_index("")
    return df


def print_confidence_intervals(class_labels, statistics):
    df = pd.DataFrame(columns=["Mean AUC (CI 5%-95%)"])
    for i in range(len(class_labels)):
        mean = statistics.mean(axis=1)[i]
        max_ = np.quantile(statistics, .95, axis=1)[i]
        min_ = np.quantile(statistics, .05, axis=1)[i]
        df.loc[class_labels[i]] = ["%.2f (%.2f-%.2f)" % (mean, min_, max_)]
    return df


def get_curve(gt, pred, target_names, curve='roc'):
    for i in range(len(target_names)):
        if curve == 'roc':
            curve_function = roc_curve
            auc_roc = roc_auc_score(gt[:, i], pred[:, i])
            label = target_names[i] + " AUC: %.3f " % auc_roc
            xlabel = "False positive rate"
            ylabel = "True positive rate"
            a, b, _ = curve_function(gt[:, i], pred[:, i])
            plt.figure(1, figsize=(7, 7))
            plt.plot([0, 1], [0, 1], 'k--')
            plt.plot(a, b, label=label)
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)

            plt.legend(loc='upper center', bbox_to_anchor=(1.3, 1),
                       fancybox=True, ncol=1)
        elif curve == 'prc':
            precision, recall, _ = precision_recall_curve(gt[:, i], pred[:, i])
            average_precision = average_precision_score(gt[:, i], pred[:, i])
            label = target_names[i] + " Avg.: %.3f " % average_precision
            plt.figure(1, figsize=(7, 7))
            plt.step(recall, precision, where='post', label=label)
            plt.xlabel('Recall')
            plt.ylabel('Precision')
            plt.ylim([0.0, 1.05])
            plt.xlim([0.0, 1.0])
            plt.legend(loc='upper center', bbox_to_anchor=(1.3, 1),
                       fancybox=True, ncol=1)
def get_accuracy(y, pred, th=0.5):
    """
    Compute accuracy of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        accuracy (float): accuracy of predictions at threshold
    """
    accuracy = 0.0
 
    
    # get TP, FP, TN, FN using our previously defined functions
    TP = true_positives(y, pred, th)
    FP = false_positives(y, pred, th)
    TN = true_negatives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # Compute accuracy using TP, FP, TN, FN
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    
    return accuracy

def get_prevalence(y):
    """
    Compute accuracy of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
    Returns:
        prevalence (float): prevalence of positive cases
    """
    prevalence = 0.0

    
    prevalence = np.mean(y)
    

    
    return prevalence


def get_sensitivity(y, pred, th=0.5):
 
    sensitivity = 0.0
    
    
    # get TP and FN using our previously defined functions
    TP = true_positives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # use TP and FN to compute sensitivity
    sensitivity = TP / (TP + FN)
    
 
    return sensitivity

def get_specificity(y, pred, th=0.5):

    specificity = 0.0
    
    
    # get TN and FP using our previously defined functions
    TN = true_negatives(y, pred, th)
    FP = false_positives(y, pred, th)
    
    # use TN and FP to compute specificity 
    specificity = TN / (TN + FP)

    
    return specificity

def get_ppv(y, pred, th=0.5):
 
    PPV = 0.0

    
    # get TP and FP using our previously defined functions
    TP = true_positives(y, pred, th)
    FP = false_positives(y, pred, th)

    # use TP and FP to compute PPV
    PPV = TP / (TP + FP)

    
    return PPV

def get_npv(y, pred, th=0.5):
  
    NPV = 0.0
    
    # get TN and FN using our previously defined functions
    TN = true_negatives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # use TN and FN to compute NPV
    NPV = TN / (TN + FN)

    
    return NPV

In [ ]:
#Evaluate the performance of the classifier using evaluation metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

metrics=get_performance_metrics(y, pred, labels,acc=get_accuracy, prevalence=get_prevalence,sens=get_sensitivity, spec=get_specificity,ppv=get_ppv, npv=get_npv,auc=roc_auc_score,f1=f1_score)
print(metrics)

#save the predicted output and assoicated probability to csv files.
metrics.to_csv('Predicted_Results.csv')

In [ ]:
#Visualization of indicative regions in CXRs
df = test_new_df
IMAGE_DIR = "../input/segmented-xrays/"

labels=['COVID','Normal','Viral Pneumonia']


# only show the labels with AUC
labels_to_show = np.take(labels, np.argsort(auc_rocs)[::-1])[:3]
compute_gradcam(model, 'COVID-3026.png', IMAGE_DIR, df, labels, labels_to_show)